In [2]:
from functools import partial
import os

import torch.nn as nn
from torchvision.models import vit_b_16
import torch

import argparse

from wb_data import WaterBirdsDataset, get_loader, get_transform_cub, log_data
from utils import evaluate, get_y_p

from wb_data import WaterBirdsDataset, get_loader, get_transform_cub, log_data


In [3]:
NUM_CLASSES = 2


In [4]:
model = vit_b_16(weights='DEFAULT')
model.heads.head = nn.Linear(model.heads.head.in_features, NUM_CLASSES)


In [5]:
model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [6]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

class_token True
conv_proj.weight True
conv_proj.bias True
encoder.pos_embedding True
encoder.layers.encoder_layer_0.ln_1.weight True
encoder.layers.encoder_layer_0.ln_1.bias True
encoder.layers.encoder_layer_0.self_attention.in_proj_weight True
encoder.layers.encoder_layer_0.self_attention.in_proj_bias True
encoder.layers.encoder_layer_0.self_attention.out_proj.weight True
encoder.layers.encoder_layer_0.self_attention.out_proj.bias True
encoder.layers.encoder_layer_0.ln_2.weight True
encoder.layers.encoder_layer_0.ln_2.bias True
encoder.layers.encoder_layer_0.mlp.0.weight True
encoder.layers.encoder_layer_0.mlp.0.bias True
encoder.layers.encoder_layer_0.mlp.3.weight True
encoder.layers.encoder_layer_0.mlp.3.bias True
encoder.layers.encoder_layer_1.ln_1.weight True
encoder.layers.encoder_layer_1.ln_1.bias True
encoder.layers.encoder_layer_1.self_attention.in_proj_weight True
encoder.layers.encoder_layer_1.self_attention.in_proj_bias True
encoder.layers.encoder_layer_1.self_attention.ou

In [12]:
for param in model.parameters():
    param.requires_grad = False

for param in model.heads.head.parameters():
    param.requires_grad = True

for name, module in model.named_modules():
    if "mlp" in name and isinstance(module, torch.nn.Linear):
        for param in module.parameters():
            param.requires_grad = True

 
for name, module in model.named_modules():
    if "encoder.ln" in name and isinstance(module, torch.nn.LayerNorm):
        for param in module.parameters():
            param.requires_grad = True
            
for name, param in model.named_parameters():
    if param.requires_grad:
        
            print(f"{name}: {param.requires_grad}")
    

encoder.layers.encoder_layer_0.mlp.0.weight: True
encoder.layers.encoder_layer_0.mlp.0.bias: True
encoder.layers.encoder_layer_0.mlp.3.weight: True
encoder.layers.encoder_layer_0.mlp.3.bias: True
encoder.layers.encoder_layer_1.mlp.0.weight: True
encoder.layers.encoder_layer_1.mlp.0.bias: True
encoder.layers.encoder_layer_1.mlp.3.weight: True
encoder.layers.encoder_layer_1.mlp.3.bias: True
encoder.layers.encoder_layer_2.mlp.0.weight: True
encoder.layers.encoder_layer_2.mlp.0.bias: True
encoder.layers.encoder_layer_2.mlp.3.weight: True
encoder.layers.encoder_layer_2.mlp.3.bias: True
encoder.layers.encoder_layer_3.mlp.0.weight: True
encoder.layers.encoder_layer_3.mlp.0.bias: True
encoder.layers.encoder_layer_3.mlp.3.weight: True
encoder.layers.encoder_layer_3.mlp.3.bias: True
encoder.layers.encoder_layer_4.mlp.0.weight: True
encoder.layers.encoder_layer_4.mlp.0.bias: True
encoder.layers.encoder_layer_4.mlp.3.weight: True
encoder.layers.encoder_layer_4.mlp.3.bias: True
encoder.layers.encod